In [1]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
load_dotenv()
API_KEY = os.getenv("ICONSTRUYE_API_KEY")
if not API_KEY:
    print("⚠️ ADVERTENCIA: La clave API no está definida en el archivo .env")
    API_KEY = None

REND_KEY = os.getenv("ICONSTRUYE_REND_KEY")
if not REND_KEY:
    print("⚠️ ADVERTENCIA: La clave API no está definida en el archivo .env")
    REND_KEY = None

In [3]:
def buscar_nc(IdOrgc,
              recep_desde,
              recep_hasta,
              FechaEmisDesde=None,
              FechaEmisHasta=None,
              api_version="1.0",
              RazonSocialC=None,
              FolioCompIngreso=None,
              NroDocumento=None,
              OrigenFactura=None,
              TipoFactura=None,
              TipoDocAsociado=None,
              Proveedor=None,
              EstadoAsociacion=None,
              EstadoDocumento=None,
              EstadoPago=None,
              Foliounico=None,
              FechaSistDesde=None,
              FechaSistHasta=None,
              TipoReferencia=None,
              info=None):
    
    print(f"📅 Consultando NC por fecha de recepción desde {recep_desde} hasta {recep_hasta}")
    
    if not API_KEY:
        print("❌ Error: No se puede consultar notas de crédito sin la clave API")
        return pd.DataFrame()
    
    url_base = "https://api.iconstruye.com/cvbf/api/NotasCorreccion/Buscar"
    headers = {
        "Ocp-Apim-Subscription-Key": API_KEY
    }


    params = {
        "IdOrgc": IdOrgc,
        "FechaRecepDesde": recep_desde,
        "FechaRecepHasta": recep_hasta,
        "api-version": api_version
    }

    opcionales = {
        "RazonSocialC": RazonSocialC,
        "FolioCompIngreso": FolioCompIngreso,
        "NroDocumento": NroDocumento,
        "OrigenFactura": OrigenFactura,
        "TipoFactura": TipoFactura,
        "TipoDocAsociado": TipoDocAsociado,
        "Proveedor": Proveedor,
        "EstadoAsociacion": EstadoAsociacion,
        "EstadoDocumento": EstadoDocumento,
        "EstadoPago": EstadoPago,
        "Foliounico": Foliounico,
        "FechaSistDesde": FechaSistDesde,
        "FechaSistHasta": FechaSistHasta,
        "TipoReferencia": TipoReferencia,
        "info": info,
        "FechaEmisDesde":FechaEmisDesde,
        "FechaEmisHasta":FechaEmisHasta
    }

    for key, value in opcionales.items():
        if value is not None:
            params[key] = value

    try:
        response = requests.get(url_base, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        if isinstance(data, list) and data:
            df = pd.DataFrame(data)
            print(f"📊 Total de Notas de Crédito encontradas: {len(df)}")
            return df
        else:
            print("⚠️ No se encontraron Notas de Crédito para ese rango de emisión.")
            return pd.DataFrame()

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ Error HTTP: {http_err}")
    except requests.exceptions.Timeout:
        print("⏱️ Error: tiempo de espera agotado al consultar la API.")
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Error de conexión o de API: {e}")

    return pd.DataFrame()


In [23]:
def obtener_detalle_nc(IdDoc, info=None, api_version="1.0"):
    url = "https://api.iconstruye.com/cvbf/api/NotasCorreccion/PorId"

    headers = {
        "Ocp-Apim-Subscription-Key": API_KEY
    }
    params = {
        "IdDoc": IdDoc,
        "info":info,
        "api-version": api_version
    }

    try:
        response = requests.get(url, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        detalle = response.json()
        print(detalle)

        # 👉 Normalización de la estructura
        if isinstance(detalle, list):
            detalle = detalle[0] if len(detalle) > 0 else {}
        elif isinstance(detalle, dict) and "items" in detalle:
            detalle = detalle["items"][0] if detalle["items"] else {}

        # 👉 Extracción segura
        monto_neto = detalle.get("cabecera", {}).get("totales", {}).get("neto", {}).get("montoNeto")
        monto_no_afecto = detalle.get("cabecera", {}).get("totales", {}).get("neto", {}).get("montoNoAfectoOExento")

        print(f"📄 Factura ID {IdDoc} → Neto: {monto_neto}, No Afecto: {monto_no_afecto}")

        return {
            "montoNeto": monto_neto,
            "montoNoAfectoOExento": monto_no_afecto
        }

    except requests.exceptions.HTTPError as http_err:
        print(f"❌ Error HTTP: {http_err}")
    except requests.exceptions.Timeout:
        print("⏱️ Tiempo de espera agotado.")
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Error de conexión/API: {e}")

    return {
        "montoNeto": None,
        "montoNoAfectoOExento": None
    }

In [24]:
notas = buscar_nc(-1,"2025-08-01","2025-08-31")


📅 Consultando NC por fecha de recepción desde 2025-08-01 hasta 2025-08-31
📊 Total de Notas de Crédito encontradas: 10


In [25]:
notas

,idDocumento,numDoc,folioDoc,folioUnico,tipoDocumento,nombreProveedor,rutProveedor,factAsociada,centroGestion,fechaSistema,fechaEmision,fechaEstimadaPago,fechaRecepcion,moneda,montoTotal,estadoDoc,estadoAsociacion
0,1205045,2770,1420-230,2770,Nota de Crédito Electrónica,Ecomet S.A,81163600-2,44511,1420 Unimarc PA Colina Brisas del Norte,2025-08-26T14:09:23.63,2025-08-26T00:00:00,2025-09-25T00:00:00,2025-08-26T00:00:00,$,68248.0,Totalmente Asociado,Totalmente Asociada
1,1204076,413,1418-294,413,Nota de Crédito Electrónica,TRANSPORTES Y ARRIENDOS DE GRÚAS LIMIDORO VERG...,78510030-1,12067,1418 Piso 8 Costanera Center,2025-08-21T13:02:30.01,2025-08-21T00:00:00,2025-09-20T00:00:00,2025-08-21T00:00:00,$,163625.0,Totalmente Asociado,Totalmente Asociada
2,1203474,2767,1422-42,2767,Nota de Crédito Electrónica,Ecomet S.A,81163600-2,44422,1422 Conversión S10 Maipú-Cerrillos,2025-08-19T15:01:41.033,2025-08-19T00:00:00,2025-09-18T00:00:00,2025-08-19T00:00:00,$,26413.0,Totalmente Asociado,Totalmente Asociada
3,1203398,195,1420-196,195,Nota de Crédito Electrónica,SOCIEDAD INMOBILIARIA E INVERSIONES BAILEY SPA,76258548-0,4365,1420 Unimarc PA Colina Brisas del Norte,2025-08-19T12:02:57.75,2025-08-19T00:00:00,2025-09-18T00:00:00,2025-08-19T00:00:00,$,132697.0,Totalmente Asociado,Totalmente Asociada
4,1202052,230356,1416-322,230356,Nota de Crédito Electrónica,COMERCIAL K SPA,77137860-9,1385975,1416 Santa Isabel MPV,2025-08-12T11:01:42.717,2025-08-12T00:00:00,2025-09-11T00:00:00,2025-08-12T00:00:00,$,4702.0,Totalmente Asociado,Totalmente Asociada
5,1201306,42,1416-312,42,Nota de Crédito Electrónica,TEC SOLUCIONES INTEGRALES EN HORMIG?N INDUSTRI...,76756266-7,277,1416 Santa Isabel MPV,2025-08-07T13:04:23.387,2025-08-07T00:00:00,2025-09-06T00:00:00,2025-08-07T00:00:00,$,1999200.0,Totalmente Asociado,Totalmente Asociada
6,1200982,13,1420-141,13,Nota de Crédito Electrónica,JONATHAN CARLOS MELENDEZ AGUIRRE,24059459-5,210,1420 Unimarc PA Colina Brisas del Norte,2025-08-06T12:02:18.973,2025-08-06T00:00:00,2025-09-05T00:00:00,2025-08-06T00:00:00,$,1131926.0,Totalmente Asociado,Totalmente Asociada
7,1200735,4245305,1421-52,4245305,Nota de Crédito Electrónica,IMPERIAL S.A.,76821330-5,19519568,1421 Super 10 S.A.,2025-08-05T12:04:14.19,2025-08-04T00:00:00,2025-09-04T00:00:00,2025-08-05T00:00:00,$,64950.0,Totalmente Asociado,Totalmente Asociada
8,1202552,788269,1422-39,788269,Nota de Crédito Electrónica,ELECTROCOM,96355000-6,6604810,1422 Conversión S10 Maipú-Cerrillos,2025-08-13T18:01:10.137,2025-08-01T00:00:00,2025-08-31T00:00:00,2025-08-01T00:00:00,$,99941.0,Totalmente Asociado,Totalmente Asociada
9,1200211,4245010,1420-133,4245010,Nota de Crédito Electrónica,IMPERIAL S.A.,76821330-5,19523449,1420 Unimarc PA Colina Brisas del Norte,2025-08-01T19:00:08.67,2025-08-01T00:00:00,2025-08-31T00:00:00,2025-08-01T00:00:00,$,427005.0,Totalmente Asociado,Totalmente Asociada


In [27]:
ejemplo = obtener_detalle_nc("1205045")

❌ Error HTTP: 404 Client Error: Resource Not Found for url: https://api.iconstruye.com/cvbf/api/NotasCorreccion/PorId?IdDoc=1205045&api-version=1.0


In [31]:
import requests

def obtener_rendiciones(rut_empresa, fecha_inicio, fecha_fin, subscription_key, estado=-1):
    """
    Obtiene rendiciones de la API de iConstruye
    """
    url = "https://api.iconstruye.com/rendicion/Integracion/ObtenerRendiciones"
    
    params = {
        'rutEmpresa': rut_empresa,
        'fechaInicio': fecha_inicio,
        'fechaFin': fecha_fin,
        'estado': estado
    }
    
    headers = {
        'Cache-Control': 'no-cache',
        'Ocp-Apim-Subscription-Key': subscription_key
    }
    
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# Ejemplo de uso
if __name__ == "__main__":
    subscription_key = "0740ba0a394b477286ea479867191a57"
    
    rendiciones = obtener_rendiciones(
        rut_empresa="77094096",
        fecha_inicio="2025-01-01", 
        fecha_fin="2025-09-16",
        subscription_key=subscription_key
    )
    
    if rendiciones:
        print(f"Total rendiciones: {len(rendiciones)}")
        print(rendiciones)
    else:
        print("No se pudieron obtener las rendiciones")

Total rendiciones: 1
[{'Id': 654837, 'Nombre': 'prueba icc', 'Numero': '4', 'Estado': -1, 'NombreEstado': 'Rechazada', 'ReembolsoGastos': True, 'FechaCreacion': '10-04-2025 10:08', 'FechaAprobacion': '10-04-2025 10:14', 'FechaContabilizacion': None, 'FechaPeriodoInicio': '01-04-2025', 'FechaPeriodoFin': '30-04-2025', 'Notas': '', 'NombreUnidadRendicion': 'Pruebas', 'CodigoUnidadRendicion': '9999', 'NombreUsuario': 'Frank Riffo ', 'CorreoUsuario': 'frank.riffo@iconstruye.com', 'RutUsuario': '13662244-7', 'NombreUsuarioFondo': None, 'CorreoUsuarioFondo': None, 'NombreUsuarioUnidad': 'Frank Riffo ', 'CorreoUsuarioUnidad': 'frank.riffo@iconstruye.com', 'PermiteMultiDistribucion': False, 'Monto': 2500, 'Gastos': 0, 'NombreCentroCosto': None, 'CodigoCentroCosto': None, 'NombreFondo': None, 'NumeroFondo': None, 'IdFondo': 0, 'NombreTipoFondo': None, 'NombreUsuarioCreador': 'Frank Riffo ', 'CorreoUsuarioCreador': 'frank.riffo@iconstruye.com', 'Pagada': False, 'IdPago': None, 'BeneficiarioFondo